In [ ]:
import numpy as np
import pandas as pd

import os
import time
import pprint
import math
import operator
import matplotlib.pyplot as plt
import heapq
from string import punctuation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from os import listdir
import re

#stop words
class Multinomial_Naive_Bayes:

    def fit(self,df,out):
        self.feature_set = df.columns.values
        dim = df.shape

        initial_size = dim[0]
        size = dim[1]

        df['_output'] = out
        classes = df["_output"].unique()

        dty = {}
        # columns = df.columns.values

        for uni in classes:
            dty[uni] = {}
            data = df[df['_output'] == uni].copy()[self.feature_set]

            class_size = data.shape[0]
            data.loc['total'] = data.sum()

            prob = class_size/initial_size

            denom = data.loc['total'].sum()


            for c in self.feature_set:

                sum = data.loc['total',c]

                if sum==0:
                    sum = (sum + 1) / (denom + size)
                else:
                    sum = sum / denom

                dty[uni][c] = sum

            # prob = round(prob,2)

            dty[uni]['_probability']= prob
            # dty[uni]['prob']['probability'] = prob


        self.train_data = dty

        return dty

    def predict(self,df):
        out = []

        for ind,row in df.iterrows():
            y_pred = self.predict_row(row)
            out.append(y_pred)

        return out


    def predict_row(self,df):

        features = list(df.keys())
        features = set(self.feature_set).intersection(set(features))

        classes = []

        for key in self.train_data.keys():
            log_prob = 1
            for f in features:
                log_prob *= self.train_data[key][f]

            log_prob *= self.train_data[key]['_probability']
            row = (key,round(log_prob,2))
            classes.append(row)

        classes = pd.DataFrame(classes,columns=['class','probability']).sort_values('probability', ascending=[False])
        classes.index = range(len(classes.index))

        return classes.loc[0,'class']



def read_file(dir):
        stop_words=["a","the","able","about","data", "about", "above",
                    "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone",
                    "along", "already", "also","although","always","am","among", "amongst", "amoungst",
                    "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway",
                    "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become",
                    "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below",
                    "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call",
                    "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail",
                    "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else",
                    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere",
                    "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly",
                    "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt",
                    "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him",
                    "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it",
                    "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile",
                    "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely",
                    "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere",
                    "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves",
                    "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming",
                    "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow",
                    "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the",
                    "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these",
                    "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together",
                    "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we",
                    "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
                    "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without",
                    "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

        #add to the dictionary the word which is not in stopwords
        ls =listdir(dir)
        dict_features = {}
        dict_record = []
        output_classes = []

        for folder in ls:
            files=listdir(dir+folder)

            for file in files:
                output_classes.append(folder)
                feature_dict = {}

                feature_dict['_out_class'] = folder

                f=open(dir+folder+'/'+file,'r',encoding="ISO-8859-1")
                text = f.read()

                tokens = re.compile('\w+').findall(text)
                # words = [token.lower() for token in tokens if ( token.isalpha() and len(token)>3 and token not in stop_words)]

                feature_dict['_file'] = file

                for token in tokens:
                    if token.isalpha() and len(token)>3 and token not in stop_words:
                        token.lower()

                        if token in feature_dict:
                            feature_dict[token] += 1
                        else:
                            feature_dict[token] = 1

                        if token in dict_features:
                            dict_features[token] += 1
                        else:
                            dict_features[token] = 1

                dict_record.append(feature_dict)


        a=sorted(dict_features.items(), key=operator.itemgetter(1), reverse=True)[:3000]
        s=a[:3000]

        # frequency=[i[1] for i in a]
        words=[i[0] for i in a]

        lgt = len(dict_record)
        data = np.zeros(shape=(lgt,len(words)))


        for i in range(lgt):
            dty = dict_record[i]
            # output_classes.append(dty['_out_class'])
            dty_keys = set(dty.keys()).intersection(set(words))
            for k in dty_keys:
                ind = words.index(k)
                data[i,ind] = dty[k]
            dict_record[i] = dty['_file']

        df = pd.DataFrame(data,columns=words)

        return df,np.array(output_classes)

def compare(y_true,y_pred):
    con_mat = confusion_matrix(y_true, y_pred)

    print("Confusion Matrix For Manual Naive Bayes")
    print(con_mat)
    print()

    print("Classification Report For Testing Data In Manual Naive Bayes")
    print(classification_report(y_true, y_pred))
    print()

def custom_MultinomialNB(df,out):
    custom_clf = Multinomial_Naive_Bayes()

    features = df.columns.values
    data = df.copy().values
    X_train, X_test, y_train, y_test = train_test_split(data, out, random_state=1)

    X_train = pd.DataFrame(X_train,columns=features)
    dty = custom_clf.fit(X_train,y_train)
    # pprint.pprint(dty)

    X_test = pd.DataFrame(X_test,columns=custom_clf.feature_set)
    y_pred = custom_clf.predict(X_test)


    print("Predicted Output For Testing Data In Manual Naive Bayes")
    print(y_pred)
    print()

    return y_test,y_pred

def scikit_NB(data,out):
    clf = MultinomialNB()

    x_train, x_test, y_train, y_test = train_test_split(data, out, random_state=1)

    clf.fit(x_train,y_train)

    y_pred = clf.predict(x_test)


    print("Predicted Output For Testing Data In Sklearn Naive Bayes")
    print(y_pred)
    print()

    return y_test,y_pred

df,out = read_file('C:/Users/Rohith/Downloads/20_newsgroups/')
print(df)

ytest,ypred = custom_MultinomialNB(df,out)
compare(ytest,ypred)

data = df.copy().values

ytest,ypred = scikit_NB(data,out)
compare(ytest,ypred)